In [ ]:
# https://stackoverflow.com/questions/37890898/how-to-set-env-variable-in-jupyter-notebook
# %env PYTHONPATH=/export/fs04/a12/rhuang/k2/k2/python:$PYTHONPATH # for `import k2`
# %env PYTHONPATH=/export/fs04/a12/rhuang/k2/build/temp.linux-x86_64-cpython-38/lib/:$PYTHONPATH # for `import _k2`
# # export PYTHONPATH=/export/fs04/a12/rhuang/icefall_align/:$PYTHONPATH
# # export PYTHONPATH=/export/fs04/a12/rhuang/icefall/:$PYTHONPATH
# %env PYTHONPATH=/export/fs04/a12/rhuang/icefall_align2/:$PYTHONPATH

import sys
sys.path.append("/export/fs04/a12/rhuang/k2/k2/python")
sys.path.append("/export/fs04/a12/rhuang/k2/build/temp.linux-x86_64-cpython-38/lib/")
sys.path.append("/export/fs04/a12/rhuang/icefall_align2/")
